## Pytorch

1. Dataset, DataLoader 로 데이터 전처리 모듈 선언
2. nn.Module 을 상속해 모델 선언
3. GPU 옮기기 및 하이퍼파라미터 정의
4. 훈련코드 구성
    - 모델이 gpu에 있으면 데이터도 gpu 에 있어야함 (마찬가지로 to(device) 로)
    - GPU에 옮길때는 할당을 해주어야 함
        - X = X.to(device)
5. 결과 확인

## Practice

In [8]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True) 
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
        #self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
        #return self.softmax(logits)

model = NeuralNetwork()

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = model.to(device)

In [10]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X.to(device))
        loss = loss_fn(pred, y.to(device))

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X.to(device))
            test_loss += loss_fn(pred, y.to(device)).item()
            correct += (pred.argmax(1) == y.to(device)).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return correct

In [11]:
learning_rate = 1e-3
batch_size = 64
epochs = 20

In [12]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

best_correct = -float('inf')
best_epoch = 0
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_correct = test_loop(test_dataloader, model, loss_fn)
    if test_correct > best_correct:
        ## 1. 모델을 save
        best_epoch = t

    ## Prevent overfitting so we do early stopping
    if t - best_epoch > interval:
        break

print("Done!")

Epoch 1
-------------------------------
loss: 2.304649  [    0/60000]
loss: 2.294202  [ 6400/60000]
loss: 2.287575  [12800/60000]
loss: 2.276610  [19200/60000]
loss: 2.266536  [25600/60000]
loss: 2.282603  [32000/60000]
loss: 2.253023  [38400/60000]
loss: 2.243524  [44800/60000]
loss: 2.225875  [51200/60000]
loss: 2.227046  [57600/60000]
Test Error: 
 Accuracy: 40.1%, Avg loss: 0.034691 

Epoch 2
-------------------------------
loss: 2.183432  [    0/60000]
loss: 2.190336  [ 6400/60000]
loss: 2.171289  [12800/60000]
loss: 2.188579  [19200/60000]
loss: 2.154699  [25600/60000]
loss: 2.117106  [32000/60000]
loss: 2.145584  [38400/60000]
loss: 2.096020  [44800/60000]
loss: 2.087687  [51200/60000]
loss: 1.990192  [57600/60000]
Test Error: 
 Accuracy: 50.5%, Avg loss: 0.032366 

Epoch 3
-------------------------------
loss: 2.046099  [    0/60000]
loss: 2.027740  [ 6400/60000]
loss: 2.083007  [12800/60000]
loss: 1.941570  [19200/60000]
loss: 2.024612  [25600/60000]
loss: 1.948576  [32000/600

### Model improvements?

1. Transfer learning (조금 더 좋은 출발조건)
2. Data augmentation (rotation, tranlation, flipping)
3. CNN
4. Hyperparameter fine-tuning
